<a href="https://colab.research.google.com/github/amulyagupta1278/Coding-Projects/blob/Deep-Learning/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Neural Networks from scratch


In [ ]:
2+2

In [ ]:
import torchvision 
import torch
from torch import nn

## Data

MNIST DATASET

In [ ]:
mnist_train_images = torchvision.datasets.MNIST(root = '', train = True, download = True, transform = torchvision.transforms.ToTensor())
mnist_test_images = torchvision.datasets.MNIST(root = '', train = False, download = True, transform = torchvision.transforms.ToTensor())

len(mnist_train_images),len(mnist_test_images)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw



(60000, 10000)

In [ ]:
mnist_train_images[0][0].shape

torch.Size([1, 28, 28])

In [ ]:
#how do you visualize data
from torchvision import transforms
im = transforms.ToPILImage()(mnist_train_images[0][0]).convert("RGB")
im

In [ ]:
#we have our dataset, lets now convert this into dataloaders
from torch.utils.data import DataLoader
train_dl = DataLoader(mnist_train_images, batch_size = 16, shuffle = True)
test_dl = DataLoader(mnist_test_images, batch_size = 32, shuffle = False)

In [ ]:
x,y = next(iter(train_dl))
x.shape, y

(torch.Size([16, 1, 28, 28]),
 tensor([0, 4, 3, 9, 0, 6, 9, 9, 4, 0, 4, 5, 2, 7, 8, 7]))

In [ ]:
def get_dls(train_ds, test_ds, bs=1):
    train_dl = DataLoader(train_ds, batch_size = bs, shuffle = True)
    test_dl = DataLoader(test_ds, batch_size = bs*2, shuffle = False)

    return train_dl, test_dl

## Model

Neural Networks


Multi-Layer Perceptron





y = w3(w2(A(w1x + b1)) + b2) + b3

y = w3w2w1x + (w3w2b1 + w3b2 + b3)

Y = Wx + B


---

y1 = (w1x + b1)
y2 = w2(Ay1) + b2
y3 = w3(Ay2) + b3


A?? Non-linear function

ReLU Function (Rectified Linear Unit)

ReLU(x) = max(x,0)



In [ ]:
class ReLU(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return torch.max(x,0)

# relu_fn = ReLU()
# relu_fn(x)

torch.return_types.max(...)

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, n_in, n_out): #dunder init
        super().__init__()
        self.layers = [nn.Linear(n_in,50), nn.ReLU(), nn.Linear(50, n_out)]
        #convert these layers into "modules", which is a PyTorch data structure that PyTorch will keep track of during learning. 
        for k,v in enumerate(self.layers): 
            self.add_module(f'layer_{k}', v)

    def forward(self,x):
        for layer in self._modules.values(): x = layer(x)
        return x

Loss Function

(softmax function)

100 - e^100 -> e^100/(sum of all exponentials)

10 - e^10

20

30

40

-50  - e^-50

Explore the cross entropy loss function

In [ ]:
def logsumexp(y): 
    #y= y/max(y) = e
    m = y.max(-1,keepdim=True).values
    return m + (y-m).exp().sum(-1,keepdim=True)

def log_softmax(y): return y-logsumexp(y)

def nll(pred, target): 
    log_sm_pred = log_softmax(pred)
    return -log_sm_pred[range(log_sm_pred.shape[0]), target].mean()

#cross entropy loss function


In [ ]:
import torch
def accuracy(pred, targ): return (torch.argmax(pred,dim=1)==targ).float().mean()

In [ ]:
class Optimizer(): #standard gradient descent optimizer
    def __init__(self, params, lr = 0.1):
        self.params = list(params)
        self.lr = lr

    def step(self):
        with torch.no_grad():
            for p in self.params: p-=self.lr*p.grad
        
    def zero_grad(self):
        for p in self.params: p.grad.data.zero_()

In [ ]:
def mnist_preprocess(x):
    x = x.view(-1,1*28*28) # 1,28,28
    x = torch.nn.functional.normalize(x)
    return x

In [ ]:
x,y = next(iter(train_dl))
print(x.shape)
x= mnist_preprocess(x)
print(x.shape)

torch.Size([16, 1, 28, 28])
torch.Size([16, 784])


In [ ]:
# ??nn.Module

In [ ]:
class Learner(): #takes in inputs, runs the training process, and tells us how accurate the model is at each step
    def __init__(self, train_dl, valid_dl, model = MLPModel(1*28*28,10), optimizer = None):
        self.train_dl = train_dl
        self.valid_dl = valid_dl
        self.model = model
        self.opt = Optimizer(self.model.parameters()) if not optimizer else optimizer

    
    def fit(self, epochs = 1, lr = None, loss_func = nll): #nll = cross_entropy function
        if lr: self.opt.lr = lr 
        for epoch in range(epochs):
            for x,y in train_dl:
                x = mnist_preprocess(x)
                pred = model(x)
                loss = nll(pred,y)
                #calculate gradients
                loss.backward()
                #parameter update
                self.opt.step()
                self.opt.zero_grad()
            
            #valid accuracy
            acc = 0.
            i=0
            for x,y in self.valid_dl:
                x = mnist_preprocess(x)
                with torch.no_grad():
                    acc+=accuracy(self.model(x),y)
                i+=1
            print(f"epoch {epoch+1}: validation accuracy: {acc/i}")

    def predict(self,x):
        x = mnist_preprocess(x)
        return torch.argmax(self.model(x),dim=1)

In [ ]:
model = MLPModel(1*28*28,10)
learn = Learner(train_dl, test_dl, model = model)

In [ ]:
model

MLPModel(
  (layer_0): Linear(in_features=784, out_features=50, bias=True)
  (layer_1): ReLU()
  (layer_2): Linear(in_features=50, out_features=10, bias=True)
)

In [ ]:
learn.fit(10, lr = 0.01)

epoch 1: validation accuracy: 0.09584664553403854
epoch 2: validation accuracy: 0.3525359332561493
epoch 3: validation accuracy: 0.8129991888999939
epoch 4: validation accuracy: 0.8613218665122986
epoch 5: validation accuracy: 0.8829872012138367
epoch 6: validation accuracy: 0.8933706283569336
epoch 7: validation accuracy: 0.8983626365661621
epoch 8: validation accuracy: 0.9013578295707703
epoch 9: validation accuracy: 0.9056509733200073
epoch 10: validation accuracy: 0.9093450307846069
